# Level 11: Advanced ORM Features

This notebook explores some of the most powerful features of the SQLAlchemy ORM, including defining relationships between models, controlling how related data is loaded, and performing more complex queries.

### Setup
To demonstrate relationships, we need a more complex schema. We'll create a `User` model and a `Post` model, where a user can have many posts (a one-to-many relationship).

In [1]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.orm import declarative_base, sessionmaker, relationship
import os

db_file = 'sqlalchemy_advanced.db'
if os.path.exists(db_file):
    os.remove(db_file)

engine = create_engine(f'sqlite:///{db_file}')
Base = declarative_base()

# Define User model
class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    posts = relationship("Post", back_populates="author")

    def __repr__(self):
        return f"<User(name='{self.name}')>"

# Define Post model
class Post(Base):
    __tablename__ = 'posts'
    id = Column(Integer, primary_key=True)
    title = Column(String)
    user_id = Column(Integer, ForeignKey('users.id'))
    author = relationship("User", back_populates="posts")

    def __repr__(self):
        return f"<Post(title='{self.title}')>"

Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()

## 11.1 Relationships

The `relationship()` function defines the link between our models. `back_populates` tells each model how to access the other side of the relationship, keeping them in sync.

In [2]:
# Create a user and some posts
user1 = User(name='Alice')
post1 = Post(title='My First Post', author=user1)
post2 = Post(title='Another Day', author=user1)

session.add(user1)
session.add_all([post1, post2])
session.commit()

In [3]:
# Accessing the relationship from the 'one' side (User -> Posts)
alice = session.query(User).filter_by(name='Alice').one()
print(f"{alice.name}'s posts: {alice.posts}")

Alice's posts: [<Post(title='My First Post')>, <Post(title='Another Day')>]


In [4]:
# Accessing the relationship from the 'many' side (Post -> User)
first_post = session.query(Post).first()
print(f"The author of '{first_post.title}' is {first_post.author}")

The author of 'My First Post' is <User(name='Alice')>


## 11.2 Eager vs. Lazy Loading

By default, SQLAlchemy uses **lazy loading**. This means that when you query for a `User`, the user's posts are *not* fetched from the database until you explicitly access the `.posts` attribute. This can lead to the **N+1 query problem**.

**Eager loading** fetches the related objects at the same time as the parent object, usually with a `JOIN`.

In [5]:
from sqlalchemy.orm import joinedload

# Use joinedload to perform an eager load
# This will issue a single SQL query with a JOIN
user_with_posts = session.query(User).options(joinedload(User.posts)).filter_by(name='Alice').one()

# Accessing .posts now does not trigger a new query
print(user_with_posts.posts)

[<Post(title='My First Post')>, <Post(title='Another Day')>]


## 11.3 Query Filtering & Chaining

The `Query` object allows for chaining multiple methods to build complex queries.

In [6]:
# Add more data for filtering
session.add_all([
    User(name='Bob', posts=[Post(title='Hello World')]),
    User(name='Charlie', posts=[Post(title='Data Science'), Post(title='SQLAlchemy')])
])
session.commit()

In [7]:
# Example of a chained query
query = (
    session.query(User)
    .filter(User.name.like('%a%')) # Names containing 'a'
    .order_by(User.name.desc()) # Order by name descending
    .limit(2) # Limit to 2 results
)

print(query.all())

[<User(name='Charlie')>, <User(name='Alice')>]


## 11.4 Aggregations in ORM

You can use functions from `sqlalchemy.sql.func` to perform aggregations.

In [8]:
from sqlalchemy import func

# Count the number of users
user_count = session.query(func.count(User.id)).scalar()
print(f"Total users: {user_count}")

Total users: 3


In [9]:
# Get the number of posts per user
posts_per_user = (
    session.query(User.name, func.count(Post.id).label('post_count'))
    .join(Post)
    .group_by(User.name)
    .all()
)

print("\nPosts per user:", posts_per_user)


Posts per user: [('Alice', 2), ('Bob', 1), ('Charlie', 2)]


## 11.5 Raw SQL in ORM

Sometimes, you need to execute a raw SQL query that is too complex for the ORM. You can do this with `session.execute()`.

In [10]:
from sqlalchemy import text

query = text("SELECT name FROM users WHERE name LIKE :search_term")
result = session.execute(query, {"search_term": "A%"})

print("Users starting with 'A':", result.fetchall())

Users starting with 'A': [('Alice',)]


In [11]:
session.close()